In [1]:
import pandas as pd 

nyt_df = pd.read_csv("data/nyt_data.csv")
washington_df = pd.read_csv("data/washington_post_with_date.csv")
cnn_df = pd.read_csv("data/cnn_data.csv")

/var/folders/zc/lk8h3bjj4bvgd74278z5plmm0000gn/T/ipykernel_34166/3808376406.py:3: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,41,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,2

In [2]:
def keep_columns(df):
    columns_to_keep = ['text', 'title', 'url', 'date']
    df = df.loc[:, columns_to_keep]
    return df

nyt_df = keep_columns(nyt_df)
print(nyt_df.columns)

washington_df = keep_columns(washington_df)
print(washington_df.columns)

cnn_df = keep_columns(cnn_df)
print(cnn_df.columns)

Index(['text', 'title', 'url', 'date'], dtype='object')
Index(['text', 'title', 'url', 'date'], dtype='object')
Index(['text', 'title', 'url', 'date'], dtype='object')


In [5]:
nyt_df['date'] = pd.to_datetime(nyt_df['date']).dt.strftime('%Y-%m-%d')
print(nyt_df['date'])

washington_df['date'] = pd.to_datetime(washington_df['date']).dt.strftime('%Y-%m-%d')
print(washington_df['date'])

0        2016-10-09
1        2016-10-06
2        2016-10-05
3        2016-10-07
4        2016-10-06
            ...    
54767    2017-07-21
54768    2017-06-15
54769    2017-08-03
54770    2017-10-06
54771    2017-07-10
Name: date, Length: 54772, dtype: object
0        2016-10-01
1        2016-10-01
2        2016-10-01
3        2016-10-01
4        2016-10-01
            ...    
99455    2017-04-24
99456    2017-11-06
99457    2017-06-16
99458    2017-04-20
99459    2017-12-04
Name: date, Length: 99460, dtype: object


In [6]:
def is_iso8601_format(value):
    try:
        pd.to_datetime(value, format='%Y-%m-%dT%H:%M:%S.%fZ')
        return True
    except ValueError:
        return False

cnn_df['date'] = cnn_df['date'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%dT%H:%M:%S.%fZ').strftime('%Y-%m-%d') if is_iso8601_format(x) else x)
print(cnn_df['date'])

0        2016-10-31
1        2017-06-30
2        2017-01-04
3        2017-06-30
4        2017-03-31
            ...    
16143    2017-07-12
16144    2016-11-10
16145    2017-08-14
16146    2017-10-17
16147    2017-07-12
Name: date, Length: 16148, dtype: object


In [4]:
from transformers import pipeline, AutoTokenizer
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
revision = "af0f99b"
classifier = pipeline("sentiment-analysis", model=model_name, revision=revision)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

/Users/rsurridge/adv_ml/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/rsurridge/adv_ml/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
from transformers import logging as transformers_logging
transformers_logging.set_verbosity_error()

def sentiment(text):
    model_name = "distilbert-base-uncased-finetuned-sst-2-english"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    classifier = pipeline("sentiment-analysis", model=model_name)

    # Adjustments for special tokens
    max_length = 512 - 10  # Ensure space for special tokens
    stride = 256

    tokens = tokenizer(text, add_special_tokens=False, return_tensors="pt", truncation=False)["input_ids"].squeeze()
    
    if len(tokens) <= max_length:
        results = [classifier(text)]  # Directly classify if within limit
    
    else:
        chunks = [tokens[i : i + max_length] for i in range(0, len(tokens), max_length - stride)]
        results = []
            
        for chunk in chunks:
            
            chunk_text = tokenizer.decode(chunk, skip_special_tokens=True)
            result = classifier(chunk_text)
            results.append(result)

    positive_scores = sum([r[0]['score'] for r in results if r[0]['label'] == 'POSITIVE'])
    negative_scores = sum([r[0]['score'] for r in results if r[0]['label'] == 'NEGATIVE'])

    return 1 if positive_scores >= negative_scores else 0

In [6]:
print(nyt_df.loc[0][0])
print("Sentiment:", sentiment(nyt_df.iloc[0][0]))

/var/folders/zc/lk8h3bjj4bvgd74278z5plmm0000gn/T/ipykernel_88375/575451433.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(nyt_df.loc[0][0])
/var/folders/zc/lk8h3bjj4bvgd74278z5plmm0000gn/T/ipykernel_88375/575451433.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("Sentiment:", sentiment(nyt_df.loc[0][0]))


https://www.nytimes.com/by/james-wagner
Sentiment: 0


In [7]:
print(nyt_df.loc[2][0])
print("Sentiment:", sentiment(nyt_df.iloc[2][0]))

http://www.nytimes.com/by/kirk-semple


/var/folders/zc/lk8h3bjj4bvgd74278z5plmm0000gn/T/ipykernel_88375/633247761.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(nyt_df.loc[2][0])
/var/folders/zc/lk8h3bjj4bvgd74278z5plmm0000gn/T/ipykernel_88375/633247761.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("Sentiment:", sentiment(nyt_df.loc[2][0]))


Sentiment: 0


In [8]:
nyt_df.loc[:, "Positive"] = nyt_df.loc[:, 'text'].apply(sentiment)
print(nyt_df.loc[:, "Positive"].value_counts())
nyt_df.to_csv('data/nyt_with_sentiment.csv', index=False)

In [9]:
washington_df.loc[:, "Positive"] = washington_df.loc[:, 'text'].apply(sentiment)
washington_df.loc[:, "Positive"].value_counts()
washington_df.to_csv('data/washington_with_sentiment.csv', index=False)

Positive
0    72826
1    26634
Name: count, dtype: int64

In [10]:
cnn_df.loc[:, "Positive"] = cnn_df.loc[:, 'text'].apply(sentiment)
cnn_df.loc[:, "Positive"].value_counts()
cnn_df.to_csv('data/cnn_with_sentiment.csv', index=False)